# Benchmarks

In [1]:
# the notebook imports
import matplotlib.pyplot as plt
import numpy as np
# this is the convenience function
from autokoopman import auto_koopman
# for a complete example, let's create an example dataset using an included benchmark system
from autokoopman.benchmark import bio2, fhn, lalo20, prde20, robe21, spring, pendulum
from glop import Glop
import random
import copy

from sklearn.metrics import mean_squared_error
import statistics
import os
import csv
import time

In [2]:
benches = [bio2.Bio2(), fhn.FitzHughNagumo(), lalo20.LaubLoomis(), pendulum.PendulumWithInput(beta=0.05), prde20.ProdDestr(), robe21.RobBench(), spring.Spring()]

In [3]:
def get_training_data(bench, param_dict, iterat):
    init_states = get_init_states(bench, param_dict["train_size"], iterat)
    if bench._input_vars:
        params = np.random.rand(param_dict["train_size"], 3) * 2 - 1
        steps = [make_input_step(*p, bench.teval) for p in params]
        training_data = bench.solve_ivps(initial_states=init_states, inputs=steps, teval=bench.teval)
    else:
        training_data = bench.solve_ivps(initial_states=init_states,tspan=[0.0, 10.0], 
                                         sampling_period=param_dict["samp_period"])
        
    return training_data

In [4]:
def get_init_states(bench, size, iterat=0):
    if hasattr(bench, 'init_constrs'):
        init_states = []
        for i in range(size):
            init_state_dict = glop_init_states(bench, i+iterat)
            init_state = []
            for name in bench.names:
                init_state.append(init_state_dict[name])
            init_states.append(init_state)
        init_states = np.array(init_states)  
    else:
        init_states = np.random.uniform(low=bench.init_set_low, 
                    high=bench.init_set_high, size=(size, len(bench.names)))
        
    print(init_states)
    return init_states

In [5]:
def glop_init_states(bench, seed):    
    constrs = []
    for constr in bench.init_constrs:
        constrs.append(constr)
    for i, (name, init_low, init_high) in enumerate(zip(bench.names, bench.init_set_low, bench.init_set_high)):
        low_constr = f"{name} >= {init_low}"
        high_constr = f"{name} <= {init_high}"
        constrs.extend([low_constr, high_constr])
        
    glop = Glop(bench.names, constrs)
    pop_item = random.randrange(len(bench.names))
    names, init_set_low, init_set_high = copy.deepcopy(bench.names), copy.deepcopy(bench.init_set_low), copy.deepcopy(bench.init_set_high)
    names.pop(pop_item)
    init_set_low.pop(pop_item)
    init_set_high.pop(pop_item)
    for i, (name, init_low, init_high) in enumerate(zip(names, init_set_low, init_set_high)):
        glop.add_tend_value_obj_fn(name, [init_low, init_high], seed+i)
    
    glop.minimize()

    sol_dict = glop.get_all_sols()    
    return sol_dict

In [6]:
def get_trajectories(bench, iv, samp_period):
    # get the model from the experiment results
    model = experiment_results['tuned_model']

    if bench._input_vars:
        test_inp = np.sin(np.linspace(0, 10, 200))

        # simulate using the learned model
        trajectory = model.solve_ivp(
            initial_state=iv,
            inputs=test_inp,
            teval=bench.teval,
        )
        # simulate the ground truth for comparison
        true_trajectory = bench.solve_ivp(
            initial_state=iv,
            inputs=test_inp,
            teval=bench.teval,
        )
        
    else:
        # simulate using the learned model
        trajectory = model.solve_ivp(
            initial_state=iv,
            tspan=(0.0, 10.0),
            sampling_period=samp_period
        )
        # simulate the ground truth for comparison
        true_trajectory = bench.solve_ivp(
            initial_state=iv,
            tspan=(0.0, 10.0),
            sampling_period=samp_period
        )
    
    return trajectory, true_trajectory

In [7]:
def test_trajectories(bench, num_tests, samp_period, iterat=0):
    mses = []
    mses_dim = [[] for x in range(len(bench.names))]
    for j in range(num_tests):
        iv = get_init_states(bench, 1, iterat+j+10000)[0]
        trajectory, true_trajectory = get_trajectories(bench, iv, samp_period)
        mse = mean_squared_error(trajectory.states.T, true_trajectory.states.T)
        mses.append(mse)
        
        for traj_dim, (trajectory, true_trajectory) in enumerate(zip(trajectory.states.T, true_trajectory.states.T)):
            mse = mean_squared_error(trajectory, true_trajectory)
            mses_dim[traj_dim].append(mse)
            
    return mses, mses_dim

In [8]:
def make_input_step(duty, on_amplitude, off_amplitude, teval):
    """produce a step response input signal for the pendulum"""
    length = len(teval)
    inp = np.zeros((length,))
    phase_idx = int(length * duty)
    inp[:phase_idx] = on_amplitude
    inp[phase_idx:] = off_amplitude
    return inp

In [9]:
def store_data(bench_name, dim, param_name, param_values, train_times, all_mses, all_mses_dim):
    if not os.path.exists('data'):
        os.makedirs('data')
        
    with open(f'data/{bench_name}', 'a') as f:
        writer = csv.writer(f)
        row = [param_name, "train_time", "Avg mse"]
        for i in range(dim):
              row.append(f'Avg mse dim {i+1}')
        writer.writerow(row)
              
        for param_value, train_time, mse, mses_dim in zip(param_values, train_times, all_mses, all_mses_dim):
              row = [param_value, train_time, mse]
              for mses in mses_dim:
                  row.append(mses)
              writer.writerow(row)
        writer.writerow([])

In [10]:
def plot(trajectory, true_trajectory):
    plt.figure(figsize=(10, 6))
    # plot the results
    plt.plot(trajectory.states.T[0], trajectory.states.T[1],label='Trajectory Prediction')
    plt.plot(true_trajectory.states.T[0], true_trajectory.states.T[1],label='Ground Truth')

    plt.xlabel("$x_1$")
    plt.ylabel("$x_2$")
    plt.grid()
    plt.legend()
    plt.title("Bio2 Test Trajectory Plot")
    plt.show()

In [11]:
all_param_values = {
    "train_size":[5, 10, 25, 50, 75, 100, 125, 150, 175, 200],
    "samp_period":[0.01, 0.025, 0.05, 0.1, 0.2, 0.25, 0.5, 1, 2],
    "obs_type":["rff","quadratic", "id"],
    "opt":["grid", "monte-carlo"],
    "n_obs":[5, 10, 25, 50, 100, 200, 300, 400, 500, 1000],
    "grid_param_slices":[5, 10, 25, 50, 100],
    "n_splits":[2, 5, 10],
    "rank":[(1, 200, 40)]
}

In [ ]:
for bench in benches:
    for param, param_values in all_param_values.items():
        all_mses = []
        all_mses_dim = []
        times = []
        param_dict = {"train_size":100,"samp_period":0.1,"obs_type":"rff","opt":"grid","n_obs":200,
                      "grid_param_slices":5,"n_splits":5,"rank":(1, 200, 40)}
        for i, param_value in enumerate(param_values):
            np.random.seed(0)
            param_dict[param] = param_value
            start = time.time()
            mses = [-1]
            iterat = 0
            while mses==[-1] and iterat < 5:
                iterat +=1
                all_mses_dim.append([])
                try:
                    # generate training data
                    print("traininggggg:")
                    training_data = get_training_data(bench, param_dict, iterat)
                    # learn model from data
                    experiment_results = auto_koopman(
                        training_data,          # list of trajectories
                        sampling_period=param_dict["samp_period"],    # sampling period of trajectory snapshots
                        obs_type=param_dict["obs_type"],         # use Random Fourier Features Observables
                        opt=param_dict["opt"],             # grid search to find best hyperparameters
                        n_obs=param_dict["n_obs"],              # maximum number of observables to try
                        max_opt_iter=200,       # maximum number of optimization iterations
                        grid_param_slices=param_dict["grid_param_slices"],# for grid search, number of slices for each parameter
                        n_splits=param_dict["n_splits"],             # k-folds validation for tuning, helps stabilize the scoring
                        rank=param_dict["rank"]       # rank range (start, stop, step) DMD hyperparameter
                    )
            
                    print("testing:")
                    mses, mses_dim = test_trajectories(bench, 10, param_dict["samp_period"], iterat)
                except ValueError:
                    print("mses: ", mses)
                    print("benchmark did not compute for this setting")
                                
            end = time.time()       
            times.append(round(end - start, 3))
            all_mses_dim.append([])
            print("The average mean square error is ", statistics.mean(mses))
            all_mses.append(statistics.mean(mses))
            for traj_dim, mses in enumerate(mses_dim):
                all_mses_dim[i].append(statistics.mean(mses))
                print(f"The average mean square error for dim {traj_dim+1} is", statistics.mean(mses))

        store_data(bench.name, len(bench.names), param, param_values, times, all_mses, all_mses_dim)

traininggggg:
[[1.00097627 1.00430379 1.00205527 1.00089766 0.9984731  1.00291788
  0.99875174 1.00783546 1.00927326]
 [0.99766883 1.0058345  1.0005779  1.00136089 1.00851193 0.99142072
  0.99174259 0.99040437 1.0066524 ]
 [1.00556314 1.00740024 1.00957237 1.00598317 0.99922959 1.00561058
  0.99236549 1.00279842 0.99286707]
 [1.00889338 1.00043697 0.99829324 0.99529111 1.00548467 0.99912301
  1.00136868 0.9903758  1.00235271]
 [1.00224191 1.00233868 1.00887496 1.00363641 0.99719016 0.99874064
  1.00395262 0.99120451 1.00333533]]


  8%|███▌                                        | 2/25 [00:00<00:06,  3.49it/s]/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2559: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2560: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
 12%|█████▎                                      | 3/25 [00:00<00:06,  3.22it/s]/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: overflow encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: invalid value encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/scipy/interpolate/_interpolate.py:641: RuntimeWarning: overflow encount

testing:
[[1.00440509 1.00612809 1.00514491 0.9916349  1.00822214 1.00439511
  1.00160416 0.99693598 1.00344277]]
[[1.00866094 0.99603868 1.00885095 1.00659257 1.00567353 1.00323501
  0.99188485 0.99585727 0.99594594]]
[[1.00499638 1.0019937  0.99465205 0.99237056 0.99005424 1.00264878
  1.00598618 0.99226064 1.00645937]]
[[0.99988215 1.00592063 1.00560591 0.99053395 0.99693341 1.00956653
  1.00010612 0.99077989 1.0028149 ]]
[[1.00426462 0.99114455 0.99022373 1.00882096 0.99264349 1.00259871
  0.99032872 0.99217375 0.99279503]]
[[0.99191499 1.00813831 0.99513429 1.0082374  1.00682551 1.00485437
  1.00047024 0.9963312  1.00030232]]
[[0.99331574 1.00029843 1.0038051  1.0010925  0.99908009 0.99966546
  0.99641131 0.9994147  0.99813003]]
[[0.99763877 0.99671178 0.99677548 1.00490011 1.00734853 1.00320022
  0.9965151  1.00308905 1.00081371]]
[[0.99856223 1.00341216 1.00346357 1.00210764 1.00430303 1.00038048
  0.99880034 1.00220488 1.00393181]]
[[1.0012344  1.00918257 0.99981579 0.9930467  

 12%|█████▎                                      | 3/25 [00:01<00:12,  1.79it/s]/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2559: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2560: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
 16%|███████                                     | 4/25 [00:02<00:11,  1.81it/s]/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: overflow encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: invalid value encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/scipy/interpolate/_interpolate.py:641: RuntimeWarning: overflow encount

testing:
[[0.99264905 0.99215389 0.99827171 0.99335382 1.00072504 0.99969696
  1.0059738  0.99022164 1.00112368]]
[[0.9976687  0.99658727 0.99585626 0.99807254 0.99928207 0.99711355
  1.00815515 0.9978489  0.99870772]]
[[1.00721003 1.00822803 0.99573046 1.00549523 0.99039079 1.00975316
  1.00687081 0.99318788 1.00861222]]
[[1.00440266 1.00133847 1.00544342 1.00979454 1.00405025 0.9962379
  1.0083203  1.00753203 0.99031931]]
[[1.00289533 0.99344659 1.00263698 0.99213158 1.00855485 1.00539582
  0.99075399 1.00321249 1.0001619 ]]
[[0.99165072 1.0018641  0.99023237 0.99487573 0.99840397 0.99225376
  0.99148732 0.99526324 0.99858154]]
[[0.99766768 0.99358725 1.00269468 0.99015006 1.0082782  1.00012995
  1.00763475 0.99174073 1.00114824]]
[[1.00313321 1.00027624 0.99422735 1.00242138 1.00812239 1.00176571
  0.99261462 1.0074435  1.00488639]]
[[1.00349847 0.99909359 0.99543973 1.00390598 0.99451664 0.99089765
  1.0054914  0.99356476 1.00572294]]
[[0.99721277 1.00640614 0.99438034 1.00581569 1

 12%|█████▎                                      | 3/25 [00:03<00:26,  1.19s/it]/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2559: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2560: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
100%|███████████████████████████████████████████| 25/25 [00:30<00:00,  1.21s/it]


testing:
[[1.00118907 0.99892698 0.99872555 0.99321302 0.99594717 1.00708339
  1.00654985 0.99449636 0.99238271]]
[[1.00780045 0.99335021 0.9981709  0.99400424 0.99252867 0.99647244
  1.00939792 0.99032564 0.99720699]]
[[0.99540632 1.00770464 0.99223461 0.99521126 0.99645193 0.99008585
  0.99702405 0.99184182 1.00098323]]
[[0.99377738 1.00901134 0.99339252 0.99676902 0.99457612 1.001925
  0.99209038 1.00952707 0.99960039]]
[[0.99073752 1.00602275 1.00655621 0.99603618 1.00131541 1.00066815
  0.99877044 1.0054499  1.00143861]]
[[1.00307381 0.9925826  1.0000937  1.00667577 0.99926526 0.99693585
  0.99880115 1.00103907 0.99341096]]
[[1.00620484 1.00214379 1.00931569 1.00117669 0.99889258 1.00184923
  1.00815885 0.99048134 0.99113919]]
[[1.00187809 0.99567071 0.99882862 0.99165436 0.99290516 1.00354989
  0.99174724 0.99466645 1.00591058]]
[[0.99658582 1.00501803 1.00135413 1.00861457 1.0056584  0.99235123
  0.99133474 0.99515249 0.99818956]]
[[0.99431732 0.99304473 1.00591148 0.99791798 0.

 12%|█████▎                                      | 3/25 [00:06<00:46,  2.11s/it]/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2559: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2560: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
 80%|██████████████████████████████████▍        | 20/25 [00:42<00:10,  2.19s/it]